# Plain-Text CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [2]:
import pandas as pd  # data mangling and transforming
import numpy as np  # handling vectors and matrices

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


### dataset

In [3]:
train_X = np.load(dir_data+'train_Xu.npy')
train_y = np.load(dir_data+'train_yu.npy')

In [4]:
# # use only the next day
# train_y = train_y[:,:1]

In [5]:
print(train_X.shape)
print(train_y.shape)

(994, 7, 1)
(994, 7)


## Modeling

### Options

In [6]:
# net hyper parameters
epochs = 20
batch_size = 4
lr = 0.001
# dimensions
n_timesteps = train_X.shape[1] 
n_features = train_X.shape[2]
n_outputs = train_y.shape[1] 

### Tensorflow

In [7]:
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(n_outputs))
model.compile(loss='mse', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [9]:
# final predictions
y_tf = model.predict(train_X)

### Pytorch

In [10]:
# define class
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(n_features, 16, 3, stride=1)
        self.l1 = nn.Linear(16*2, 10)
        self.l2 = nn.Linear(10, n_outputs)
        
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool1d(x, kernel_size=2)
        x = x.view(-1, 16*2) # flatten
        x = F.relu(self.l1(x))
        x = self.l2(x)
        return x

In [11]:
nn_model = Net()

In [12]:
opt = torch.optim.Adam(nn_model.parameters(), lr=lr)
loss_fn = F.mse_loss

In [13]:
# transform data to tensors (& change order of channels and n_features)
inputs = torch.from_numpy(np.array(train_X.reshape(train_X.shape[0], 
                                                   train_X.shape[2], 
                                                   train_X.shape[1]), dtype='float32'))
targets = torch.from_numpy(np.array(train_y, dtype='float32'))
train_Xy = DataLoader(TensorDataset(inputs, targets), 
                      batch_size, 
                      shuffle=False)

In [14]:
# Define a utility function to train the model
def fit(df, num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in df:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            opt.zero_grad()
            loss.backward()
            opt.step()
        if epoch%5==0:
            print("Epoch %s: Loss %s"%(epoch, loss.data.item()))
    print('Training loss: ', loss_fn(model(inputs), targets).data.item())

In [15]:
# Train the model for X epochs
fit(train_Xy, epochs, nn_model, loss_fn, opt)

Epoch 0: Loss 213628.90625
Epoch 5: Loss 207416.03125
Epoch 10: Loss 205578.71875
Epoch 15: Loss 204630.78125
Training loss:  280163.25


In [16]:
y_pt = nn_model(inputs)

#### Comparison

In [17]:
print('Pytorch RMSE: %s'%np.sqrt(loss_fn(y_pt, targets).data.item()))

Pytorch RMSE: 529.3044964857185


In [18]:
print('Tensorflow RMSE: %s'%np.sqrt(loss_fn(torch.from_numpy(np.array(y_tf, dtype='float32')), targets).data.item()))

Tensorflow RMSE: 527.6184404946438
